# Disruptive Papers in Fetal Surgery

This notebook:
- Reads search terms from text files
- Builds search hedges for each term list and writes them to TXT
- Builds full search queries from each set of hedges and writes them to TXT
- Pulls search results from PubMed and writes them to CSV
- Tests search results against lists of PMIDs identified as papers to include or exclude
- Maps PMIDs to MAG IDs via OpenAlex API
- Pulls citation counts, development, and disruption scores from the AggregatedMAG.txt dataset
- exports scoring data to CSV and XLSX
- optionally creates datestamped snapshot of input/output files

TODO:
- more logging
- function documentation
- finish test_results function
- MeSH analysis of search results/test articles

## Import Libraries

In [ ]:
import glob
import logging
import pandas as pd
from pathlib import Path
from pymed import PubMed
import requests
import shutil
from styleframe import StyleFrame, Styler, utils
import sys
import time
from tqdm import tqdm

## Define Constants and Create Directories

In [ ]:
# API parameters
email = "whimar@oshu.edu"
pubmed = PubMed(tool="FetalSurgerySearch", email=email)
openalex = "https://api.openalex.org/works?per-page=100&filter=pmid:"
headers = {"user-agent": "mailto:" + email}
max_results = 100000

datestamp = time.strftime("%Y%m%d")
TOP_PAPERS = 100
MAG_SCORES = "AggregatedMAG.txt"

LOG_DIR = "log/"
TEST_DIR = "tests/*"

# data diectory paths
DATA_DIR = "data/"
JOURNAL_DIR = DATA_DIR + "journals/*"
KEYWORD_DIR = DATA_DIR + "keywords/*"
MESH_DIR = DATA_DIR + "mesh-terms/*"

# output directory paths
TARGET_DIR = "target/"
CSV_DIR = TARGET_DIR + "csv/"
HEDGE_DIR = TARGET_DIR + "hedges/"
QUERY_DIR = TARGET_DIR + "queries/"
XLSX_DIR = TARGET_DIR + "xlsx/"

# archive directory
ARCHIVE_DIR = "archive/"

# create output directories if they don't exist
Path(TARGET_DIR).mkdir(parents=True, exist_ok=True)
Path(CSV_DIR).mkdir(parents=True, exist_ok=True)
Path(HEDGE_DIR).mkdir(parents=True, exist_ok=True)
Path(LOG_DIR).mkdir(parents=True, exist_ok=True)
Path(QUERY_DIR).mkdir(parents=True, exist_ok=True)
Path(XLSX_DIR).mkdir(parents=True, exist_ok=True)
Path(ARCHIVE_DIR).mkdir(parents=True, exist_ok=True)

## Configure Logging

In [ ]:
logging.basicConfig(
    filename=f"{LOG_DIR}{datestamp}-hedgebill.log",
    filemode="w",
    force=True,
    format="%(asctime)s.%(msecs)03d : %(levelname)s : %(message)s",
    level=logging.INFO,
    datefmt="%Y-%m-%d %H:%M:%S",
)
logger = logging.getLogger()

## Function Definitions

### create_snapshot

In [ ]:
def create_snapshot():
    today_folder = f"{ARCHIVE_DIR}{datestamp}/"
    data_folder = f"{ARCHIVE_DIR}{datestamp}/data/"

    Path(today_folder).mkdir(parents=True, exist_ok=True)
    Path(data_folder).mkdir(parents=True, exist_ok=True)

    shutil.copytree(DATA_DIR, data_folder, dirs_exist_ok=True)

    if Path(TARGET_DIR).isdir():
        shutil.move(TARGET_DIR, today_folder)

### pmid_to_magid

In [ ]:
def pmid_to_magid(articles_df, querycount, querytype):
    magsfile = f"{CSV_DIR}{datestamp}-{querycount}-{querytype}-MAGID.csv"
    magsfilepath = Path(magsfile)

    # Have we already pulled these MAG IDs?
    if magsfilepath.is_file():
        # If so, read the results we saved to CSV
        logger.info("Reading MAGID data from file")
        mags_df = pd.read_csv(magsfile, index_col="long_pmid")
    else:
        logger.info("Pulling new MAGIDs from OpenAlex")
        # If not, pull MAGIDs from OpenAlex
        pmids = articles_df["pmid"].tolist()
        pmid_groups = [pmids[i : i + 50] for i in range(0, len(pmids), 50)]

        pmid_strings = []
        mags = {}
        for pmid_group in pmid_groups:
            pmid_strings.append("|".join(map(str, pmid_group)))

        for pmid_string in tqdm(pmid_strings, desc="openalex"):
            response = requests.get(openalex + pmid_string, headers=headers)
            if response.status_code == 200:
                works = response.json()
                for work in works["results"]:
                    if "mag" in work["ids"]:
                        mags[work["ids"]["pmid"]] = work["ids"]["mag"]
            else:
                logger.error("Error making openalex request")

        mags_df = pd.DataFrame.from_dict(mags, orient="index", columns=["magid"])
        mags_df.index.name = "long_pmid"
        mags_df.sort_index(inplace=True)
        mags_df.to_csv(magsfile)
        mags_df = pd.read_csv(magsfile, index_col="long_pmid")

    return mags_df

### run_query

In [ ]:
def run_query(query, querycount, querytype):
    resultsfile = f"{CSV_DIR}{datestamp}-{querycount}-{querytype}-results.csv"
    samplefile = f"{CSV_DIR}{datestamp}-{querycount}-{querytype}-SAMPLE.csv"
    resultsfilepath = Path(resultsfile)

    # Have we already run this query?
    if resultsfilepath.is_file():
        # If so, read the results we saved to CSV
        logger.info("Reading query results from file")
        articles_df = pd.read_csv(resultsfile, index_col="long_pmid")
    else:
        # If not, run the query
        logger.info("Running new PubMed query")
        queryresults = pubmed.query(query, max_results=max_results)
        # drop results that are not journal articles
        articles = [
            [
                article.pubmed_id.splitlines()[0],
                article.title,
                article.journal,
                article.publication_date,
                article.doi,
                article.abstract,
            ]
            for article in tqdm(queryresults, total=querycount, desc=querytype)
            if hasattr(article, "journal")
        ]
        articlecount = len(articles)
        articles_df = pd.DataFrame(
            articles,
            columns=["pmid", "title", "journal", "pubdate", "doi", "abstract"],
        )

        # add columns of helpful URLs
        # hyperlinked URLs makes things more accessible in the XLSX output
        articles_df["doi"] = [
            f'=HYPERLINK("https://doi.org/{doi}")' if doi is not None else ""
            for doi in articles_df["doi"]
        ]
        articles_df["pubmed"] = [
            f'=HYPERLINK("https://pubmed.ncbi.nlm.nih.gov/{pmid}", {pmid})'
            for pmid in articles_df["pmid"]
        ]
        articles_df["ohsu_library"] = [
            f'=HYPERLINK("https://librarysearch.ohsu.edu/openurl/OHSU/OHSU?sid=Entrez:PubMed&id=pmid:{pmid}", "Find @ OHSU")'
            for pmid in articles_df["pmid"]
        ]
        articles_df["rush_library"] = [
            f'=HYPERLINK("https://i-share-rsh.primo.exlibrisgroup.com/openurl/01CARLI_RSH/01CARLI_RSH:CARLI_RSH?sid=Entrez:PubMed&id=pmid:{pmid}", "Find @ Rush")'
            for pmid in articles_df["pmid"]
        ]
        articles_df["long_pmid"] = [
            f"https://pubmed.ncbi.nlm.nih.gov/{pmid}" for pmid in articles_df["pmid"]
        ]

        articles_df.set_index("long_pmid", inplace=True)
        articles_df.sort_index(inplace=True)

        articles_df.to_csv(resultsfile)
        articles_df.sample(n=25).to_csv(samplefile)
        articles_df = pd.read_csv(resultsfile, index_col="long_pmid")

    return articles_df

### score_results

In [ ]:
def score_results(articles_df, querycount, querytype):
    citefile = f"{CSV_DIR}{datestamp}-{querycount}-{querytype}-topcited.csv"
    developfile = f"{CSV_DIR}{datestamp}-{querycount}-{querytype}-topdevelopmental.csv"
    disruptfile = f"{CSV_DIR}{datestamp}-{querycount}-{querytype}-topdisruptive.csv"

    mag_df = pd.read_csv(
        f"AggregatedMAG.txt",
        sep="\t",
        usecols=[0, 5, 6],
        names=["magid", "num_citations", "disruption_score"],
        index_col="magid",
        dtype={"citation": "int64", "disruption": "float64"},
    )
    mag_df.sort_index(inplace=True)

    scored_df = articles_df.join(mag_df)
    scored_df.reset_index(inplace=True)

    cited_df = scored_df[
        [
            "num_citations",
            "title",
            "journal",
            "abstract",
            "pubdate",
            "magid",
            "pmid",
            "doi",
            "pubmed",
            "ohsu_library",
            "rush_library",
        ]
    ].nlargest(TOP_PAPERS, columns="num_citations")
    development_df = scored_df[
        [
            "disruption_score",
            "title",
            "journal",
            "abstract",
            "pubdate",
            "magid",
            "pmid",
            "doi",
            "pubmed",
            "ohsu_library",
            "rush_library",
        ]
    ].nsmallest(TOP_PAPERS, columns="disruption_score")
    disrupt_sf = scored_df[
        [
            "disruption_score",
            "title",
            "journal",
            "abstract",
            "pubdate",
            "magid",
            "pmid",
            "doi",
            "pubmed",
            "ohsu_library",
            "rush_library",
        ]
    ].nlargest(TOP_PAPERS, columns="disruption_score")

    cited_df.to_csv(
        citefile,
        index=False,
        columns=[
            "num_citations",
            "magid",
            "pmid",
            "title",
            "journal",
            "pubdate",
            "doi",
            "abstract",
        ],
    )
    development_df.to_csv(
        developfile,
        index=False,
        columns=[
            "disruption_score",
            "magid",
            "pmid",
            "title",
            "journal",
            "pubdate",
            "doi",
            "abstract",
        ],
    )
    disrupt_sf.to_csv(
        disruptfile,
        index=False,
        columns=[
            "disruption_score",
            "magid",
            "pmid",
            "title",
            "journal",
            "pubdate",
            "doi",
            "abstract",
        ],
    )

    return cited_df, development_df, disrupt_sf

### style_output

In [ ]:
def style_output(cited_df, develop_df, disrupt_sf, query, querycount, querytype):
    scorefile = f"{XLSX_DIR}{datestamp}-{querycount}-{querytype}-scores.xlsx"

    cited_sf = StyleFrame(cited_df.drop("pmid", axis=1))
    develop_sf = StyleFrame(develop_df.drop("pmid", axis=1))
    disrupt_sf = StyleFrame(disrupt_sf.drop("pmid", axis=1))
    query_s = pd.Series({f"{querytype} query": query})

    default_style = Styler(
        font=utils.fonts.calibri,
        font_size=11,
        border_type=utils.borders.default_grid,
        horizontal_alignment=utils.horizontal_alignments.left,
        wrap_text=False,
        shrink_to_fit=False,
    )

    header_style = Styler(
        bg_color=utils.colors.black,
        bold=True,
        font=utils.fonts.calibri,
        font_color=utils.colors.white,
        font_size=14,
        horizontal_alignment=utils.horizontal_alignments.left,
        shrink_to_fit=False,
        wrap_text=False,
        vertical_alignment=utils.vertical_alignments.center,
    )
    hyperlink_style = Styler(
        font_color=utils.colors.blue,
        protection=True,
        underline=utils.underline.single,
    )
    float_style = Styler(
        number_format="0.000000000000",
        horizontal_alignment=utils.horizontal_alignments.right,
    )

    cited_sf.set_column_width_dict(
        col_width_dict={
            ("pubdate", "magid", "pubmed"): 12,
            ("num_citations", "ohsu_library", "rush_library"): 15,
            ("title", "journal", "abstract", "doi"): 50,
        }
    )
    cited_sf.apply_headers_style(header_style)
    cited_sf.apply_column_style(cited_sf.columns, styler_obj=default_style)
    cited_sf.apply_column_style(
        ["doi", "pubmed", "ohsu_library", "rush_library"],
        styler_obj=Styler.combine(default_style, hyperlink_style),
    )

    develop_sf.set_column_width_dict(
        col_width_dict={
            ("pubdate", "magid", "pubmed"): 12,
            ("ohsu_library", "rush_library"): 15,
            ("disruption_score"): 20,
            ("title", "journal", "abstract", "doi"): 50,
        }
    )
    develop_sf.apply_headers_style(header_style)
    develop_sf.apply_column_style(develop_sf.columns, styler_obj=default_style)
    develop_sf.apply_column_style(
        "disruption_score", styler_obj=Styler.combine(default_style, float_style)
    )
    develop_sf.apply_column_style(
        ["doi", "pubmed", "ohsu_library", "rush_library"],
        styler_obj=Styler.combine(default_style, hyperlink_style),
    )

    disrupt_sf.set_column_width_dict(
        col_width_dict={
            ("pubdate", "magid", "pubmed"): 12,
            ("ohsu_library", "rush_library"): 15,
            ("disruption_score"): 20,
            ("title", "journal", "abstract", "doi"): 50,
        }
    )
    disrupt_sf.apply_headers_style(header_style)
    disrupt_sf.apply_column_style(disrupt_sf.columns, styler_obj=default_style)
    disrupt_sf.apply_column_style(
        "disruption_score", styler_obj=Styler.combine(default_style, float_style)
    )
    disrupt_sf.apply_column_style(
        ["doi", "pubmed", "ohsu_library", "rush_library"],
        styler_obj=Styler.combine(default_style, hyperlink_style),
    )

    with pd.ExcelWriter(scorefile) as sfile:
        cited_sf.to_excel(
            sfile,
            index=False,
            columns=[
                "num_citations",
                "title",
                "journal",
                "abstract",
                "pubdate",
                "magid",
                "doi",
                "pubmed",
                "ohsu_library",
                "rush_library",
            ],
            sheet_name="top cited",
        )
        develop_sf.to_excel(
            sfile,
            index=False,
            columns=[
                "disruption_score",
                "title",
                "journal",
                "abstract",
                "pubdate",
                "magid",
                "doi",
                "pubmed",
                "ohsu_library",
                "rush_library",
            ],
            sheet_name="top developmental",
        )
        disrupt_sf.to_excel(
            sfile,
            index=False,
            columns=[
                "disruption_score",
                "title",
                "journal",
                "abstract",
                "pubdate",
                "magid",
                "doi",
                "pubmed",
                "ohsu_library",
                "rush_library",
            ],
            sheet_name="top disruptive",
        )
        query_s.to_excel(sfile, index=False, header=False, sheet_name="pubmed query")

### test_results

In [ ]:
def test_results(articles_df):
    # test_results_file = f"{XLS}{}"
    tests = {}
    results = {}
    coverage = {}
    testfiles = sorted(glob.glob(TEST_DIR))

    for testfile in testfiles:
        logger.info("test file: %s", testfile)
        tfile = Path(testfile)
        tests[tfile.stem] = pd.read_csv(testfile, names=["pmid"])
        results[tfile.stem] = tests[tfile.stem].merge(
            articles_df.reset_index(drop=True), how="inner", on="pmid"
        )
        coverage[tfile.stem] = (
            str(len(results[tfile.stem]) / len(tests[tfile.stem]) * 100) + "%"
        )

    # save these to a file? or return results and coverage to pass into the XSLX?
    # tests["exclude"].merge(mesh_articles_df.reset_index(drop=True), how="inner", on="pmid")
    # tests["include"].merge(mesh_articles_df.reset_index(drop=True), how="inner", on="pmid")

### write_hedge

In [ ]:
def write_hedge(hedge, building_block, hedgetype):
    hedgecount = str(pubmed.getTotalResultsCount(hedge))
    hedgefile = f"{HEDGE_DIR}{datestamp}-{building_block}-{hedgecount}-{hedgetype}.txt"
    with open(hedgefile, "w+") as hfile:
        hfile.write(f"({hedge})")
    logger.info("building block: %s", building_block)
    logger.info("hedge type: %s", hedgetype)
    logger.info("hedge result count: %s", hedgecount)
    logger.info("hedge file: %s", hedgefile)
    logger.info("hedge: %s", hedge)

### write_query

In [ ]:
def write_query(query, querytype):
    try:
        querycount = pubmed.getTotalResultsCount(query)
    except requests.exceptions.HTTPError as httperr:
        logger.error(httperr)
        queryfile = f"{QUERY_DIR}{datestamp}-{querycount}-TOOLONG.txt"
        with open(queryfile, "w+") as qfile:
            qfile.write(query)
        sys.exit(1)
    except Exception as e:
        logger.error("e.message")
        sys.exit(1)
    else:
        queryfile = f"{QUERY_DIR}{datestamp}-{str(querycount)}-{querytype}.txt"
        with open(queryfile, "w+") as qfile:
            qfile.write(query)
        logger.info("query type: %s", querytype)
        logger.info("query result count: %d", querycount)
        logger.info("query file: %s", queryfile)
        logger.info("query: %s", query)
        return querycount

## Build Hedges, Run Queries, Pull Scores, Write Outputs

### MeSH Query

In [ ]:
mesh_querytype = "[mesh]"
mesh_hedges = []
mesh_termfiles = sorted(glob.glob(MESH_DIR))

for mesh_termfile in mesh_termfiles:
    logger.info("term file: %s", mesh_termfile)
    building_block = Path(mesh_termfile).stem
    with open(mesh_termfile, "r") as tfile:
        mesh_hedge = " OR ".join(sorted(tfile.read().strip().split("\n")))
        write_hedge(mesh_hedge, building_block, mesh_querytype)
        mesh_hedges.append(f"({mesh_hedge})")

mesh_query = " AND ".join(mesh_hedges)
mesh_querycount = write_query(mesh_query, mesh_querytype)

mesh_articles_df = run_query(mesh_query, mesh_querycount, mesh_querytype)
# test_results(mesh_articles_df)

mesh_mags_df = pmid_to_magid(mesh_articles_df, mesh_querycount, mesh_querytype)

mesh_df = mesh_mags_df.join(mesh_articles_df).set_index("magid").sort_index()
mesh_df.index = mesh_df.index.astype("int64")

mesh_cited_df, mesh_develop_df, mesh_disrupt_df = score_results(
    mesh_df, mesh_querycount, mesh_querytype
)

style_output(
    mesh_cited_df,
    mesh_develop_df,
    mesh_disrupt_df,
    mesh_query,
    mesh_querycount,
    mesh_querytype,
)

In [ ]:
# create_snapshot()